In [7]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical', 
                                            shuffle=False)  # Ensure that it doesn't shuffle the data

Found 624 images belonging to 2 classes.


In [9]:
vgg = VGG16(input_shape=[224,224] + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
model.load_weights('training_1/cp.ckpt')

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
test_loss, test_accuracy = model.evaluate(test_set, steps=len(test_set), batch_size=8)

20/20 [==============================] - 14s 430ms/step - loss: 0.3711 - accuracy: 0.9231


In [13]:
predictions = model.predict(test_set, steps=len(test_set))

20/20 [==============================] - 6s 275ms/step


In [14]:
true_labels = test_set.classes


In [15]:
predicted_labels = np.argmax(predictions, axis=1)

In [16]:
report = classification_report(true_labels, predicted_labels, target_names=test_set.class_indices.keys())
confusion = confusion_matrix(true_labels, predicted_labels)

# Print the test accuracy and other metrics
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
print('\nClassification Report:\n', report)
print('\nConfusion Matrix:\n', confusion)

Test Loss: 0.3711
Test Accuracy: 0.9231

Classification Report:
               precision    recall  f1-score   support

      NORMAL       0.92      0.87      0.89       234
   PNEUMONIA       0.92      0.96      0.94       390

    accuracy                           0.92       624
   macro avg       0.92      0.91      0.92       624
weighted avg       0.92      0.92      0.92       624


Confusion Matrix:
 [[203  31]
 [ 17 373]]
